# Import Libraries

In [183]:
import os
import pandas as pd
import numpy as np
import cv2

# Load Dataset

In [184]:
df = pd.read_csv("billboard/train_labels.csv")

In [185]:
df.head()

,ID,Labels
0,9c2c67a8bab25b17.jpg,115 91 634 354 billboard 815 371 917 484 billb...
1,bee6a98eeb1ee48e.jpg,276 164 771 735 billboard
2,4ccbcbf33650b75a.jpg,121 267 484 652 billboard
3,b5c6ed3a56eb38c1.jpg,74 451 167 665 billboard 329 240 757 476 billb...
4,00a2d2f1b206b096.jpg,0 599 19 632 billboard 103 599 209 639 billboa...


# Flow

In [186]:
boxes = df['Labels'][0].split('billboard')
boxes.remove('')
boxes

['115 91 634 354 ', ' 815 371 917 484 ']

In [187]:
bounding_box = [ list(map(int,box.split())) for box in boxes]

In [188]:
get_bounding_box = lambda label: [list(map(int, box.split())) for box in label.split('billboard') if box]

df['Bounding Boxes'] = df['Labels'].apply(get_bounding_box)

In [189]:
df.head()

,ID,Labels,Bounding Boxes
0,9c2c67a8bab25b17.jpg,115 91 634 354 billboard 815 371 917 484 billb...,"[[115, 91, 634, 354], [815, 371, 917, 484]]"
1,bee6a98eeb1ee48e.jpg,276 164 771 735 billboard,"[[276, 164, 771, 735]]"
2,4ccbcbf33650b75a.jpg,121 267 484 652 billboard,"[[121, 267, 484, 652]]"
3,b5c6ed3a56eb38c1.jpg,74 451 167 665 billboard 329 240 757 476 billb...,"[[74, 451, 167, 665], [329, 240, 757, 476]]"
4,00a2d2f1b206b096.jpg,0 599 19 632 billboard 103 599 209 639 billboa...,"[[0, 599, 19, 632], [103, 599, 209, 639], [226..."


# Apply the Normalization

In [190]:
def convert_to_yolo_bbox(bbox, image_width, image_height):
    xmin, ymin, xmax, ymax = bbox
    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin

    # Normalize the values between 0 and 1
    x_center /= image_width
    y_center /= image_height
    width /= image_width
    height /= image_height

    normalized_bounding_box = [round(x_center,6), round(y_center,6), round(width,6), round(height,6)]
    return normalized_bounding_box

In [191]:
df['Image_Height_Width'] = df['ID'].apply(lambda x: cv2.imread(f"billboard/Images/{x}").shape[:2])

In [192]:
df.head()

,ID,Labels,Bounding Boxes,Image_Height_Width
0,9c2c67a8bab25b17.jpg,115 91 634 354 billboard 815 371 917 484 billb...,"[[115, 91, 634, 354], [815, 371, 917, 484]]","(683, 1024)"
1,bee6a98eeb1ee48e.jpg,276 164 771 735 billboard,"[[276, 164, 771, 735]]","(768, 1024)"
2,4ccbcbf33650b75a.jpg,121 267 484 652 billboard,"[[121, 267, 484, 652]]","(683, 1024)"
3,b5c6ed3a56eb38c1.jpg,74 451 167 665 billboard 329 240 757 476 billb...,"[[74, 451, 167, 665], [329, 240, 757, 476]]","(768, 1024)"
4,00a2d2f1b206b096.jpg,0 599 19 632 billboard 103 599 209 639 billboa...,"[[0, 599, 19, 632], [103, 599, 209, 639], [226...","(768, 1024)"


In [193]:
for index, row in df.iterrows():
    image_height, image_width = df['Image_Height_Width'][index]
    yolo_bboxes = [convert_to_yolo_bbox(box, image_width, image_height) for box in row['Bounding Boxes']]
    df.at[index, 'Bounding Boxes'] = yolo_bboxes

In [194]:
df.head()

,ID,Labels,Bounding Boxes,Image_Height_Width
0,9c2c67a8bab25b17.jpg,115 91 634 354 billboard 815 371 917 484 billb...,"[[0.365723, 0.325769, 0.506836, 0.385066], [0....","(683, 1024)"
1,bee6a98eeb1ee48e.jpg,276 164 771 735 billboard,"[[0.51123, 0.585286, 0.483398, 0.74349]]","(768, 1024)"
2,4ccbcbf33650b75a.jpg,121 267 484 652 billboard,"[[0.29541, 0.672767, 0.354492, 0.56369]]","(683, 1024)"
3,b5c6ed3a56eb38c1.jpg,74 451 167 665 billboard 329 240 757 476 billb...,"[[0.117676, 0.726562, 0.09082, 0.278646], [0.5...","(768, 1024)"
4,00a2d2f1b206b096.jpg,0 599 19 632 billboard 103 599 209 639 billboa...,"[[0.009277, 0.801432, 0.018555, 0.042969], [0....","(768, 1024)"


# Saving in respective files

In [ ]:
for index,row in df.iterrows():
        
    # File path
    file = df['ID'][index].split('.')[0]
    file_path = f"C:/Users/Ghost/Desktop/galli_maps/training_billboard/billboard/Images_and_Labels/{file}.txt"

    bounding_boxes_with_zeros = [[0] + box for box in df['Bounding Boxes'][index]]
    
    # Open the file in write mode
    with open(file_path, 'w') as file:
        # Write each sublist to a new line in the file
        for box in bounding_boxes_with_zeros:
            line = ' '.join(map(str, box)) + '\n'  # Convert the sublist to a string and add a newline character
            file.write(line)

    # print("File saved successfully.")